# Planted Noisy kXOR: Kikuchi Adjacency Matrix

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `KikuchiMatrixEntry`
Adjacency matrix oracle for the Kikuchi matrix.

Given a kXOR instance $\mathcal{I}$ with $n$ variables, $m$ constraints,
the Kikuchi matrix with parameter $\ell$ is indexed by ${[n] \choose l}$.
For $S, T \in {[n] \choose l}$, the entry is given by
$H_{S, T} = B_{\mathcal{I}}(S \Delta T)/M$, where $M$ is the max entry.

This bloq implements the transform:
    $$
    |0 \rangle |S\rangle |T\rangle
    \mapsto
    (\sqrt{H_{S, T}}|0\rangle + \sqrt{1 - |H_{S, T}|}|1\rangle)|S\rangle |T\rangle
    $$

This is equivalent to $O_H$ (Def. 4.3) from the paper, but is optimized to classically
compute the `arccos` of the entries, and directly apply the rotation,
instead of computing them using a quantum circuit.

This bloq performs the following steps
1. Compute the symmetric difference $D = S \Delta T$.
2. Compute the index $j$ s.t. $U_j = D$ (where $U_j$ are a list of unique scopes)
4. Apply a controlled Y-rotation with angle for the $j$-th entry.
5. Uncompute steps 3, 2, 1.

#### Parameters
 - `inst`: k-XOR instance
 - `ell`: the Kikuchi parameter $\ell$, must be a multiple of $k$.
 - `entry_bitsize`: number of bits to approximate each rotation angle to.
 - `cv`: single bit control value (0 or 1), or None for uncontrolled (default). 

#### Registers
 - `S`: row index
 - `T`: column index
 - `q`: the qubit to rotate by $Ry(2 \arccos(\sqrt{H_{S,T} / M}))$ as defined above. 

#### References
 - [Quartic quantum speedups for planted inference](https://arxiv.org/abs/2406.19378v1). Definition 4.3. Theorem 4.17 para 3.


In [ ]:
from qualtran.bloqs.optimization.k_xor_sat import KikuchiMatrixEntry

### Example Instances

In [ ]:
from qualtran.bloqs.optimization.k_xor_sat.kxor_instance import KXorInstance

n, m, k, c = sympy.symbols("n m k c", positive=True, integer=True)
inst = KXorInstance.symbolic(n=n, m=m, k=k)
ell = c * k

kikuchi_matrix_entry_symb = KikuchiMatrixEntry(inst, ell, entry_bitsize=3)

In [ ]:
from qualtran.bloqs.optimization.k_xor_sat.kxor_instance import example_kxor_instance

inst = example_kxor_instance()
ell = 8

kikuchi_matrix_entry = KikuchiMatrixEntry(inst, ell, entry_bitsize=3)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([kikuchi_matrix_entry_symb, kikuchi_matrix_entry],
           ['`kikuchi_matrix_entry_symb`', '`kikuchi_matrix_entry`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
kikuchi_matrix_entry_symb_g, kikuchi_matrix_entry_symb_sigma = kikuchi_matrix_entry_symb.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(kikuchi_matrix_entry_symb_g)
show_counts_sigma(kikuchi_matrix_entry_symb_sigma)